# Agent IA : Extraction et Analyse de Transcriptions YouTube avec GPT-4o

Ce document explique le fonctionnement d'un agent conversationnel intelligent qui peut :

* Extraire automatiquement la transcription d'une vidéo YouTube
* Utiliser cette transcription pour répondre à des questions en langage naturel
* S'appuyer sur le modèle GPT-4o pour une compréhension plus poussée et des réponses précises



##  Outils et bibliothèques utilisés

| Outil / Librairie        | Rôle                                                                          |
| ------------------------ | ----------------------------------------------------------------------------- |
| `youtube_transcript_api` | Récupération des sous-titres (transcriptions) depuis une URL YouTube          |
| `agents`                 | Création de l'agent, déclaration de la fonction outil, streaming des réponses |
| `dotenv`                 | Chargement de la clé OpenAI depuis un fichier `.env` pour plus de sécurité    |
| `openai.types.responses` | Gestion des réponses à flux continu avec `ResponseTextDeltaEvent`             |
| `asyncio`                | Gestion de la boucle de dialogue de façon non bloquante                       |


##  Etapes clés du fonctionnement de l'agent

### 1. Chargement des variables d'environnement

On utilise `load_dotenv()` pour charger la clé API OpenAI (
`OPENAI_API_KEY`) depuis un fichier `.env`. Cela évite de stocker des clés sensibles en dur dans le code.


### 2. Définition de la fonction outil : `fetch_youtube_transcript()`

* Reçoit une URL YouTube
* Extrait l'identifiant de la vidéo
* Utilise `.fetch()` pour récupérer les sous-titres
* Formate chaque entrée avec un timestamp `[MM:SS]`

Cette fonction est décorée avec `@function_tool` pour être utilisable par l'agent automatiquement.



### 3. Création de l'agent avec `Agent()`

L'agent est créé avec :

* Un nom
* Des instructions : d'abord récupérer la transcription, puis répondre aux questions
* L'outil `fetch_youtube_transcript`
* Le modèle **GPT-4o**, qui permet de gérer des contextes longs et répondre de façon naturelle


### 4. Boucle de dialogue interactive

L'utilisateur peut saisir :

* Une question textuelle (ex : "De quoi parle la vidéo ?")
* Une URL de vidéo YouTube

L'agent gère les deux cas automatiquement. La transcription est appelée si besoin, puis stockée.





In [ ]:
!pip install jupyterlab ipykernel ipywidgets openai-agents openai python-dotenv youtube-transcript-api textstat

###  Chargement des bibliothèques essentielles

Dans cette section, nous importons toutes les bibliothèques nécessaires au bon fonctionnement de notre agent intelligent :

- `re` : pour manipuler les expressions régulières (extraction de l'ID vidéo YouTube)
- `asyncio` : pour exécuter l'application de manière asynchrone, sans blocage
- `dotenv` : pour charger automatiquement la clé API OpenAI depuis un fichier `.env` sécurisé
- `youtube_transcript_api` : pour récupérer les sous-titres (transcription) des vidéos YouTube
- `youtube_transcript_api._errors` : pour gérer proprement les erreurs spécifiques à YouTube (ex : vidéo privée, transcription absente, etc.)
- `agents` : module local définissant la structure de l’agent, les outils utilisables, et l’exécution des dialogues
- `openai.types.responses` : pour gérer les réponses `streamées` du modèle (affichage en temps réel mot par mot)




In [7]:
import re                                # Utilisé pour extraire l'ID de la vidéo à partir de l'URL via expressions régulières
import asyncio                           # Permet la gestion asynchrone du dialogue avec l'utilisateur
from dotenv import load_dotenv           # Charge les variables d'environnement depuis un fichier .env (clé API OpenAI)

from youtube_transcript_api import YouTubeTranscriptApi              # Fournit l'accès aux transcriptions des vidéos YouTube
from youtube_transcript_api._errors import (                         # Importe les erreurs spécifiques liées aux vidéos
    NoTranscriptFound,              # Levée si aucune transcription n'est disponible
    TranscriptsDisabled,           # Levée si l’auteur de la vidéo a désactivé les transcriptions
    VideoUnavailable,              # Levée si la vidéo est privée ou supprimée
    CouldNotRetrieveTranscript     # Levée en cas d’échec général de récupération de la transcription
)

from agents import Agent, function_tool, Runner                      # Permet de définir l’agent, l’outil utilisé et l’exécution de l’agent
from openai.types.responses import ResponseTextDeltaEvent            # Gère les réponses reçues de manière progressive (streaming)


In [8]:
import logging                                                   # Permet de configurer les niveaux de journalisation du programme
logging.getLogger("httpx").setLevel(logging.WARNING)             # Réduit les messages de log HTTPX à WARNING uniquement (supprime les logs INFO)


In [9]:
from dotenv import load_dotenv                              # Importe la fonction pour charger les variables d’environnement depuis un fichier .env
import os                                                   # Permet d’accéder aux variables d’environnement via os.getenv()

load_dotenv(dotenv_path=".env", override=True)              # Charge explicitement le fichier .env et écrase les variables existantes si nécessaire

api_key = os.getenv("OPENAI_API_KEY")                       # Récupère la clé API OpenAI depuis l’environnement
print("Clé API chargée :", api_key[:8] + "..." if api_key else "Aucune clé détectée")  # Affiche un extrait de la clé ou un message d’erreur


Clé API chargée : sk-proj-...


### Récupération et formatage de la transcription

Cette fonction permet d’extraire automatiquement la transcription d’une vidéo YouTube à partir de son URL.  
Elle identifie l’ID de la vidéo, interroge l’API via la méthode `.fetch()` et formate le texte avec des horodatages `[MM:SS]`.  
Elle prend également en compte les cas d’erreur courants (vidéo privée, transcription désactivée, etc.).  
Grâce au décorateur `@function_tool`, elle est intégrée comme outil accessible à l’agent conversationnel.


In [10]:
@function_tool                             # Rend cette fonction accessible à l'agent comme un outil externe
async def fetch_youtube_transcript(url: str) -> str:
    """Récupère la transcription YouTube avec la vraie API."""
    try:
        # Extraction de l'ID de la vidéo
        video_id_pattern = r'(?:v=|\/)([0-9A-Za-z_-]{11}).*'
        match = re.search(video_id_pattern, url)
        
        if not match:
            return "⚠️ URL YouTube invalide. Vérifiez le format de l'URL."
            
        video_id = match.group(1)
        print(f"🎥 Récupération de la transcription pour: {video_id}")
        
        # Créer une instance de l'API
        ytt_api = YouTubeTranscriptApi()
        transcript_data = None
        
        # Méthode 1: Utiliser fetch() avec des langues prioritaires
        try:
            transcript_data = ytt_api.fetch(video_id, languages=['fr'])
            print("✅ Transcription française récupérée")
        except NoTranscriptFound:
            try:
                transcript_data = ytt_api.fetch(video_id, languages=['en'])
                print("✅ Transcription anglaise récupérée")
            except NoTranscriptFound:
                try:
                    # Sans spécifier de langue (défaut: anglais)
                    transcript_data = ytt_api.fetch(video_id)
                    print("✅ Transcription par défaut récupérée")
                except Exception as fetch_error:
                    print(f"🔄 fetch() a échoué: {str(fetch_error)}")
                    
                    # Méthode 2: Utiliser list() puis fetch() sur l'objet transcript
                    try:
                        print("🔄 Essai avec list() puis fetch()...")
                        transcript_list = ytt_api.list(video_id)
                        
                        # Essayer de trouver une transcription française ou anglaise
                        try:
                            transcript_obj = transcript_list.find_transcript(['fr', 'en'])
                            transcript_data = transcript_obj.fetch()
                            print(f"✅ Transcription {transcript_obj.language} trouvée via list()")
                        except NoTranscriptFound:
                            # Prendre la première transcription disponible
                            try:
                                available_transcripts = list(transcript_list)
                                if available_transcripts:
                                    first_transcript = available_transcripts[0]
                                    transcript_data = first_transcript.fetch()
                                    print(f"✅ Première transcription disponible: {first_transcript.language}")
                                else:
                                    return "❌ Aucune transcription trouvée pour cette vidéo"
                            except Exception as list_fetch_error:
                                return f"❌ Erreur lors de fetch() sur transcript: {str(list_fetch_error)}"
                    except Exception as list_error:
                        return f"❌ Erreur list(): {str(list_error)}"
        
        # Vérifier que nous avons des données
        if not transcript_data:
            return "❌ Aucune donnée de transcription récupérée"
        
        # Formatage de la transcription
        if hasattr(transcript_data, 'snippets'):
            # C'est un objet FetchedTranscript avec des snippets
            formatted_transcript = []
            for snippet in transcript_data.snippets:
                try:
                    start_time = float(snippet.start)
                    minutes = int(start_time // 60)
                    seconds = int(start_time % 60)
                    timestamp = f"[{minutes:02d}:{seconds:02d}]"
                    text = snippet.text.strip()
                    if text:
                        formatted_transcript.append(f"{timestamp} {text}")
                except Exception as snippet_error:
                    print(f"⚠️ Erreur formatage snippet: {snippet_error}")
                    continue
            
            if formatted_transcript:
                result = "\n".join(formatted_transcript)
                print(f"✅ Transcription formatée: {len(formatted_transcript)} snippets")
                return result
            else:
                return "❌ Aucun snippet valide trouvé"
                
        elif isinstance(transcript_data, list):
            # Format liste de dictionnaires (ancien format)
            formatted_transcript = []
            for entry in transcript_data:
                try:
                    if isinstance(entry, dict) and 'text' in entry:
                        start_time = float(entry.get('start', 0))
                        minutes = int(start_time // 60)
                        seconds = int(start_time % 60)
                        timestamp = f"[{minutes:02d}:{seconds:02d}]"
                        text = entry['text'].strip()
                        if text:
                            formatted_transcript.append(f"{timestamp} {text}")
                except Exception as entry_error:
                    print(f"⚠️ Erreur formatage entrée: {entry_error}")
                    continue
            
            if formatted_transcript:
                result = "\n".join(formatted_transcript)
                print(f"✅ Transcription formatée: {len(formatted_transcript)} entrées")
                return result
            else:
                return "❌ Aucune entrée valide trouvée"
        else:
            return f"❌ Format de transcription non reconnu: {type(transcript_data)}"

    except TranscriptsDisabled:
        return "❌ Les transcriptions sont désactivées pour cette vidéo"
    except VideoUnavailable:
        return "❌ Cette vidéo n'est pas disponible"
    except CouldNotRetrieveTranscript:
        return "❌ Impossible de récupérer la transcription pour cette vidéo"
    except Exception as e:
        print(f"❌ Erreur inattendue: {str(e)}")
        return f"❌ Erreur inattendue: {str(e)}"
   

### Instruction précise pour l'agent

In [11]:
# Instructions de l'agent
instructions = (
    "Tu es un assistant expert en analyse de vidéos YouTube et en création de contenu. "
    "Lorsque l'utilisateur te donne une URL, commence par récupérer automatiquement la transcription de la vidéo. "
    "Analyse ensuite le contenu en identifiant les messages clés, le ton employé, les intentions de l’auteur, les cibles et les points marquants.\n\n"
    "Tu peux ensuite répondre à deux types de demandes :\n"
    "1. **Poser des questions sur la vidéo** : l'utilisateur peut te poser des questions précises pour mieux comprendre ou explorer le contenu de la vidéo.\n"
    "2. **Générer des contenus pour les réseaux sociaux** :\n"
    "   - Un **post LinkedIn** (800 à 1200 caractères), structuré, professionnel, avec une bonne accroche, un développement clair et une ouverture à l’interaction (question, call-to-action…)\n"
    "   - Un **post Instagram** (300 à 600 caractères), plus direct, percutant, avec un ton léger, inspirant ou engageant, centré sur une idée clé de la vidéo.\n\n"
    "Adapte toujours le style et le ton au format demandé. "
    "Si la vidéo contient un message central fort ou un storytelling marquant, fais-en le fil conducteur."
)


### Définition de l’agent conversationnel

L’agent est configuré à l’aide de la classe `Agent`, qui permet de définir son rôle, ses capacités et le modèle qu’il utilise.

- `name` : nom descriptif de l’agent (ici, spécialisé dans l’analyse de vidéos YouTube)
- `instructions` : consignes initiales données au modèle pour orienter son comportement
- `tools` : liste des fonctions qu’il peut appeler automatiquement en fonction des besoins (ici, `fetch_youtube_transcript`)
- `model` : choix explicite du modèle utilisé, ici **`gpt-4o`** pour profiter d’un contexte étendu et de performances optimisées

Cette définition permet à l’agent de raisonner de manière autonome et d’utiliser des outils externes si nécessaire.


In [12]:
agent = Agent(
    name="Tube2Post Agent",               # Nom de l’agent (utilisé pour l'identification ou le debug)
    instructions=instructions,                     # Consignes initiales pour orienter le comportement du modèle
    tools=[fetch_youtube_transcript],              # Liste des outils que l’agent peut utiliser (fonctions accessibles)
    model="gpt-4o"                                  # Modèle de langage utilisé (GPT-4o pour contexte étendu et performance)
)


### Lancement de la boucle de dialogue avec l'utilisateur

La fonction `main()` initialise et gère l’interaction en continu entre l'utilisateur et l'agent.

#### Fonctionnement :
- Affiche un message d’accueil et attend une entrée utilisateur
- Gère les commandes de sortie (`exit`, `quit`, etc.)
- Stocke les échanges dans une liste `input_items` pour maintenir le contexte
- Limite l’historique à 8 messages pour éviter les dépassements de capacité du modèle (token limit)
- Transmet les échanges à l’agent via `Runner.run_streamed(...)` pour obtenir une réponse en streaming
- Gère les différents types d’événements retournés : texte généré, appel d’outil, résultats d’outil
- Affiche la réponse en temps réel ligne par ligne

Cette boucle permet de simuler une véritable conversation, tout en exploitant les capacités du modèle GPT-4o et de l’outil `fetch_youtube_transcript` si nécessaire.


In [14]:
# Fonction principale de dialogue asynchrone
async def main():
    input_items = []                                              # Initialise l'historique de la conversation

    print("=== YouTube Transcript Agent ===")                     # Message d’accueil
    print("Tapez 'exit' pour quitter.")                           # Indique comment quitter
    print("Posez une question ou fournissez une URL YouTube.")    # Invite à interagir

    while True:
        try:
            user_input = await asyncio.to_thread(input, "\nYou: ")  # Saisie utilisateur dans un thread non bloquant
            user_input = user_input.strip()                          # Nettoyage de l’entrée
        except (EOFError, KeyboardInterrupt):                      # Gestion interruption clavier
            print("\nSession interrompue.")                        # Message de sortie
            break

        if user_input.lower() in ['exit', 'quit', 'bye']:          # Commande de sortie
            print("À bientôt !")                                   # Message de départ
            break

        if not user_input:
            continue                                               # Ignore les entrées vides

        input_items.append({"content": user_input, "role": "user"}) # Ajoute la question à l’historique

        input_items = input_items[-8:]                             # Limite l’historique à 8 messages max

        print("\nAgent: ", end="", flush=True)                     # Préparation de l'affichage de la réponse

        try:
            result = Runner.run_streamed(agent, input=input_items) # Exécution de l’agent avec streaming

            async for event in result.stream_events():             # Boucle sur les événements reçus
                if event.type == "raw_response_event" and isinstance(event.data, ResponseTextDeltaEvent):
                    print(event.data.delta, end="", flush=True)    # Affichage progressif du texte

                elif event.type == "run_item_stream_event":
                    if event.item.type == "tool_call_item":
                        print("\n-- Récupération de la transcription...")   # Indique que l’outil est appelé

                    elif event.item.type == "tool_call_output_item":
                        if "⚠️" in event.item.output or "❌" in event.item.output:
                            print(f"-- Erreur : {event.item.output}")       # Affiche les erreurs sans les stocker
                        else:
                            print("-- Transcription récupérée.")            # Indique succès
                            input_items.append({
                                "content": "La transcription a été récupérée avec succès.",
                                "role": "system"
                            })                                              # Stocke une note générique dans le contexte

                    elif event.item.type == "message_output_item":
                        input_items.append({
                            "content": event.item.raw_item.content[0].text,
                            "role": "assistant"
                        })                                                  # Ajoute la réponse finale au contexte

        except Exception as e:
            print(f"\nErreur pendant l'exécution : {e}")                    # Affiche les erreurs inattendues

        print("\n")                                                         # Ligne de séparation entre les échanges


# Lancement de la boucle (notebook ou script async)
await main()


=== YouTube Transcript Agent ===
Tapez 'exit' pour quitter.
Posez une question ou fournissez une URL YouTube.



You:  bye


À bientôt !


In [ ]:
# # to run in a .py script use
# if __name__ == "__main__":
#     asyncio.run(main())

In [ ]:
pip freeze > requirements.txt


In [ ]:
pip install pipreqs
